## Class 5 - Bagging and Boosting

### Recap of lecture and introductory remarks
In yesterday's lecture, we introduced bagging and boosting as two techniques to reduce variance and reduce variance of decision trees. Bagging and boosting are not specific to decision trees, but we will see them in action with this kind of model.

We used bagging to train a set of small decision trees (weak learners) on subsets of the training data, whose individual predictions we aggregate to make a single prediction. The resulting model is an _ensemble_ model. We have seen that `Random Forests` are popular learning algorithms that combine bagging with random sampling of features, to induce diversity in decision trees and further regularization.

On the other hand, boosting consists in training a sequence of decision trees which iteratively reduce the error of the previous decision tree because they are fitted on the residuals or on the gradients of the previous tree. We have focused specifically on `gradient boosting` and indicated `XGBoost` as a particularly powerful implementation of boosting + bagging.

Today, we will go back to the bike data, and fit `RandomForest` and `XGBoost` models, comparing their performances to those of models fitted previously. We will implement Random Forests using `scikit-learn`, and XGBoost using the XGBoost package: https://xgboost.readthedocs.io/en/stable/ 

**Note**: As last week, under `nbs/class_05` you will find a notebook called `example.ipynb`, where I provide an example of how to run today's exercise on sample data.

### Operational remarks
Two suggestions on how to go about this, based on where you are at regarding exercises from previous weeks.

If you have done exercises from class 2 and 3, you will have one/two notebooks with baseline, linear, KNN, and linear regularized models, as well as records of performances (which will be handy to compare performances of our new methods). In this case, my suggestion would be to work on a new notebook where you only fit the new models, and load the performance of the old models for comparison.

If you have not done exercises from previous classes, you have two options:
- Work on a new notebook and "manually" compare the performance of your new models to plots from previous weeks;
- Work on a new notebook and also implement a couple of models from previous weeks for comparison.

### Today's exercise
Work in groups on the following tasks

1. Fit a `Random Forest` model to the data (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor), using cross-validation to define the best possible range of parameters
    - There are a number of parameters that should be passed to the estimator. Carefully read the documentation, and identify a few hyperparameters you might want to manipulate
    - Define a series of possible values for these hyperparameters, and store this information into a Python dictionary. For each hyperparameter, the dictionary should include the name of the hyperparameters (as a string) as `key`, and a list including a range of possible values as `value`
    - Pass your estimator and the parameter grid to `GridSearchCV`: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html and fit this object to your training set. If you have defined *a lot* of possible values, you can consider using `RandomizedSearchCV`: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html. **Note**: you need to pass something appropriate as the value of the `scoring` argument
    - Try to answer the following questions:
        - What is `GridSearchCV` doing?
        - What is the difference between `RandomizedSearchCV` and `GridSearchCV`?
        - **Bonus question**: Given that we do have a validation set, could we do model selection without using cross-validation? Which parameter in `GridSearchCV` or `RandomizedSearchCV` would you have to change, and how, to do so?
    - Find out which hyperparameters gave the best result
        - **Hint**: look at the `.best_estimator_` attribute on a fitted `GridSearchCV`/`RandomizedSearchCV` and `.get_params()`
    - Compute the performance of this model on the training, validation, and test set
    - Compute and plot feature importances for the resulting model. You can look at the `.feature_importances_` attribute of the best estimator.
        - **Bonus question**: which method is used by default to compute feature importances? Is any other method available in `sklearn`?

2. Do the exact same things as 1., this time using `XGBRegressor` (https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBRegressor)
    - Note: you will have to install `xgboost` (https://xgboost.readthedocs.io/en/stable/install.html) to run this
    - You will have to define an appropriate `scoring` parameter
    - Parameters for grid/randomized search will be slightly different: look at the documentation for XGBRegressor, and make informed choices based on what we discussed in class

3. Plot the performance of the best Random Forest models and the best XGBoost models, against models you fitted previously
    - Which models perform best?
    - How does the performance profile of RandomForest compare to XGBoost? Why?

4. Compare feature importances across `RandomForest` and `XGBoost`: do they look similar/different?

5. Overall reflection on modeling process
    - Reflect back on your choices for previous models: should you have transformed any of the features before fitting Linear Regression, KNN, or regularized models?
    - Can you think of ways in which our predictive problem can be made more interesting from a business perspective?
    - Which aspect of the data are we *not* modeling, that we could/should model?


### Extra tasks
- Estimate a `DecisionTreeRegressor` with cross-validation, using the same logic we applied above: how does the performance of the resulting model compare to `RandomForestRegressor` and `XGBoost` regressor?
- Go back to your fitted `GridSearchCV` or `RandomizedSearchCV`, and inspect their attributes. Can you plot performance against values of each of the parameters you are fitting? Is there any systematic pattern?
- Reflect on hyperparameters passed to `GridSearchCV` or `RandomizedSearchCV`: how do you expect that individual manipulations of these parameters would affect the bias/variance profile of your models?



In [5]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree as tr
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as pltv

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

#### load data

In [15]:
bike_dat = pd.read_csv('/work/data/class_01/bikes.csv')

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


In [18]:
#X = bike_dat[['windspeed', 'temp']]
#y = bike_dat[['cnt']]

X = bike_dat.iloc[:,2:15].values
y = bike_dat.iloc[:,16].values


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, 
                                                    random_state=42) 

# let's further split the training set into a training and a validation set (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train,
                                                  test_size=X_test.shape[0] / X_train.shape[0],  
                                                  random_state=42)                                                    

In [20]:
X_train.shape

(12165, 13)

#### dict of hyper params 
- number of trees
- Complexity of the trees 
- Number of features sampled
-  Number of data points sampled 

In [21]:
params = ({'n_estimators': [50,100,1000], # number of trees 
          'criterion': ["friedman_mse", "poisson"],
          'min_samples_split': [2,4],
          'bootstrap': [False, True],
          'oob_score': [True], # this only runs when bootstrap is true
          'n_jobs': [-1],
          'random_state': [100],
          'max_features': [2,6,13]}) # remember n_features = 15... ideally we would have more 

In [32]:
from itertools import product
 
#### THE NUMBER OF COMBINATIONS IN THE GRID SEARCH 
lists = list(params.values())
combos = list(product(*lists))
len(combos) 
### This times the number of cross val (default = 5) is total number of fits 

##### grid search CV

In [22]:
from sklearn.model_selection import GridSearchCV

#### fitting the model and the params in a grid search (just meaning it combines the diff params value)
rand_for_mod = RandomForestRegressor()
grid_mod = GridSearchCV(rand_for_mod, params)

#### fitting to data
grid_mod.fit(X_train, y_train)

/work/LauraWulffPaaby#7567/DatSci_24_forked/DataSci-AU-24/venv_dat_sci24/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/work/LauraWulffPaaby#7567/DatSci_24_forked/DataSci-AU-24/venv_dat_sci24/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/work/LauraWulffPaaby#7567/DatSci_24_forked/DataSci-AU-24/venv_dat_sci24/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args,

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [False, True],
                         'criterion': ['friedman_mse', 'poisson'],
                         'max_features': [2, 6, 13],
                         'min_samples_split': [2, 4],
                         'n_estimators': [50, 100, 1000], 'n_jobs': [-1],
                         'oob_score': [True], 'random_state': [100]})

In [38]:
import pickle as pkl

file_path = 'grid_model_random_forrest.pkl'
with open(file_path, 'wb') as file:
    pkl.dump(grid_mod, file)

#### model evaluation: 
trying to find the combination of hyperparameters that gave the best model performance

then **feature importance** - find the most informative features and plot:

In [51]:
### use .best_estimator and .get_params() to find the best model and important features
best_model = grid_mod.best_estimator_

In [40]:
grid_mod.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__criterion': 'squared_error',
 'estimator__max_depth': None,
 'estimator__max_features': 1.0,
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__monotonic_cst': None,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': None,
 'estimator__oob_score': False,
 'estimator__random_state': None,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': RandomForestRegressor(),
 'n_jobs': None,
 'param_grid': {'n_estimators': [50, 100, 1000],
  'criterion': ['friedman_mse', 'poisson'],
  'min_samples_split': [2, 4],
  'bootstrap': [False, True],
  'oob_score': [True],
  'n_jobs': [-1],
  'random_state': [100],
  'max_features': [2, 6, 13]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,

In [52]:
#### performance calculator 
from sklearn.metrics import mean_squared_error, r2_score

performances = []

for x,y,nsplit in zip([X_train, X_val, X_test],
                    [y_train, y_val, y_test],
                    ['train', 'val', 'test']):
    preds = best_model.predict(x)
    r2 = r2_score(y, preds)
    performance = np.sqrt(mean_squared_error(y, preds))
    performances.append({'model': 'random_forest',
                         'split': nsplit,
                         'rmse': performance.round(4),
                         'r2': r2.round(4)})

In [53]:
performances

[{'model': 'random_forest', 'split': 'train', 'rmse': 12.7151, 'r2': 0.9952},
 {'model': 'random_forest', 'split': 'val', 'rmse': 31.3972, 'r2': 0.9675},
 {'model': 'random_forest', 'split': 'test', 'rmse': 34.7457, 'r2': 0.9619}]

In [54]:
best_model.feature_importances_

array([0.00633026, 0.04873175, 0.0133107 , 0.2402707 , 0.00110706,
       0.00615481, 0.07273861, 0.00280951, 0.00603129, 0.00582717,
       0.00709916, 0.00438389, 0.58520508])